In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/zomato/zomato.csv


In [2]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')
import re
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
zomato_real=pd.read_csv("../input/zomato/zomato.csv")
zomato_real.head()

,url,address,name,online_order,book_table,rate,votes,phone,location,rest_type,dish_liked,cuisines,approx_cost(for two people),reviews_list,menu_item,listed_in(type),listed_in(city)
0,https://www.zomato.com/bangalore/jalsa-banasha...,"942, 21st Main Road, 2nd Stage, Banashankari, ...",Jalsa,Yes,Yes,4.1/5,775,080 42297555\r\n+91 9743772233,Banashankari,Casual Dining,"Pasta, Lunch Buffet, Masala Papad, Paneer Laja...","North Indian, Mughlai, Chinese",800,"[('Rated 4.0', 'RATED\n A beautiful place to ...",[],Buffet,Banashankari
1,https://www.zomato.com/bangalore/spice-elephan...,"2nd Floor, 80 Feet Road, Near Big Bazaar, 6th ...",Spice Elephant,Yes,No,4.1/5,787,080 41714161,Banashankari,Casual Dining,"Momos, Lunch Buffet, Chocolate Nirvana, Thai G...","Chinese, North Indian, Thai",800,"[('Rated 4.0', 'RATED\n Had been here for din...",[],Buffet,Banashankari
2,https://www.zomato.com/SanchurroBangalore?cont...,"1112, Next to KIMS Medical College, 17th Cross...",San Churro Cafe,Yes,No,3.8/5,918,+91 9663487993,Banashankari,"Cafe, Casual Dining","Churros, Cannelloni, Minestrone Soup, Hot Choc...","Cafe, Mexican, Italian",800,"[('Rated 3.0', ""RATED\n Ambience is not that ...",[],Buffet,Banashankari
3,https://www.zomato.com/bangalore/addhuri-udupi...,"1st Floor, Annakuteera, 3rd Stage, Banashankar...",Addhuri Udupi Bhojana,No,No,3.7/5,88,+91 9620009302,Banashankari,Quick Bites,Masala Dosa,"South Indian, North Indian",300,"[('Rated 4.0', ""RATED\n Great food and proper...",[],Buffet,Banashankari
4,https://www.zomato.com/bangalore/grand-village...,"10, 3rd Floor, Lakshmi Associates, Gandhi Baza...",Grand Village,No,No,3.8/5,166,+91 8026612447\r\n+91 9901210005,Basavanagudi,Casual Dining,"Panipuri, Gol Gappe","North Indian, Rajasthani",600,"[('Rated 4.0', 'RATED\n Very good restaurant ...",[],Buffet,Banashankari


In [4]:
#Deleting Unnnecessary Columns
zomato=zomato_real.drop(['url','dish_liked','phone'],axis=1)
#Removing the Duplicates
zomato.duplicated().sum()
zomato.drop_duplicates(inplace=True)
#Remove the NaN values from the dataset
zomato.isnull().sum()
zomato.dropna(how='any',inplace=True)
#Changing the column names
zomato = zomato.rename(columns={'approx_cost(for two people)':'cost','listed_in(type)':'type', 'listed_in(city)':'city'})
#Some Transformations
zomato['cost'] = zomato['cost'].astype(str) #Changing the cost to string
zomato['cost'] = zomato['cost'].apply(lambda x: x.replace(',',''))
zomato['cost'] = zomato['cost'].astype(float)
#Removing '/5' from Rates
zomato = zomato.loc[zomato.rate !='NEW']
zomato = zomato.loc[zomato.rate !='-'].reset_index(drop=True)
remove_slash = lambda x: x.replace('/5', '') if type(x) == np.str else x
zomato.rate = zomato.rate.apply(remove_slash).str.strip().astype('float')

# Adjust the column names
zomato.name = zomato.name.apply(lambda x:x.title())
zomato.online_order.replace(('Yes','No'),(True, False),inplace=True)
zomato.book_table.replace(('Yes','No'),(True, False),inplace=True)

# Computing Mean Rating
restaurants = list(zomato['name'].unique())
zomato['Mean Rating'] = 0

for i in range(len(restaurants)):
    zomato['Mean Rating'][zomato['name'] == restaurants[i]] = zomato['rate'][zomato['name'] == restaurants[i]].mean()
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range = (1,5))
zomato[['Mean Rating']] = scaler.fit_transform(zomato[['Mean Rating']]).round(2)

In [5]:
# Lower Casing
zomato["reviews_list"] = zomato["reviews_list"].str.lower()

## Removal of Puctuations
import string
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_punctuation(text))

## Removal of Stopwords
from nltk.corpus import stopwords
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_stopwords(text))

## Removal of URLS
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

zomato["reviews_list"] = zomato["reviews_list"].apply(lambda text: remove_urls(text))

zomato[['reviews_list', 'cuisines']].sample(5)

,reviews_list,cuisines
32786,rated 40 ratedn got 1 rice combo 1 full meal k...,North Indian
37335,rated 40 ratedn amazing food pricegood place t...,"Fast Food, Desserts"
30763,rated 50 ratedn undoubtedly say best steak pla...,"North Eastern, Asian, Naga, Steak, Momos"
4160,rated 40 ratedn nnbelgian waffles originated s...,"Desserts, Beverages"
4173,rated 30 ratedn heard lot place hence decided ...,"Nepalese, Tibetan, Momos"


In [6]:
# RESTAURANT NAMES:
restaurant_names = list(zomato['name'].unique())
zomato1=zomato.drop(['address','rest_type', 'type', 'menu_item', 'votes'],axis=1)
import pandas
# Randomly sample 70% of your dataframe
df_percent = zomato1.sample(frac=0.7)
print(df_percent)

                                          name  online_order  book_table  \
23256                       Wok Paper Scissors          True       False   
20853                           Mini Fast Food         False       False   
37644                 Samudra Bar & Restaurant         False       False   
23158  Saara'S Grill N Spice - Ss Lumina Hotel         False        True   
34343                                Minimelts         False       False   
...                                        ...           ...         ...   
14760                                   Forage         False        True   
26189                           Lord Of Combos          True       False   
7532                               China Pearl          True        True   
21852                                   Atithi          True        True   
32732                             O2 Juice Bar          True       False   

       rate               location                                  cuisines  \
23256  

In [7]:
df_percent.set_index('name', inplace=True)
indices = pd.Series(df_percent.index)

# Creating tf-idf matrix
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=0, stop_words='english')
tfidf_matrix = tfidf.fit_transform(df_percent['reviews_list'])
print(tfidf)
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

TfidfVectorizer(min_df=0, ngram_range=(1, 2), stop_words='english')


In [8]:
print(df_percent)

                                         online_order  book_table  rate  \
name                                                                      
Wok Paper Scissors                               True       False   4.3   
Mini Fast Food                                  False       False   3.1   
Samudra Bar & Restaurant                        False       False   2.9   
Saara'S Grill N Spice - Ss Lumina Hotel         False        True   4.3   
Minimelts                                       False       False   3.9   
...                                               ...         ...   ...   
Forage                                          False        True   4.1   
Lord Of Combos                                   True       False   3.0   
China Pearl                                      True        True   4.4   
Atithi                                           True        True   3.9   
O2 Juice Bar                                     True       False   3.6   

                        

In [9]:
def recommend(name, cosine_similarities = cosine_similarities):
    
    # Create a list to put top restaurants
    recommend_restaurant = []
    
    # Find the index of the hotel entered
    idx = indices[indices == name].index[0]
    
    # Find the restaurants with a similar cosine-sim value and order them from bigges number
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending=False)
    
    top_indexes = list(score_series.iloc[0:30].index)
    for each in top_indexes:
            recommend_restaurant.append(list(df_percent.index)[each])
    list_set = set(recommend_restaurant)
    unique_list = (list(list_set))
    recommend_restaurant
    return(unique_list)
recommend('Pai Vihar')

['Shree Thali',
 'Mayura Sagar',
 'Pai Vihar',
 'Punjabi Tasty Khana',
 'Cinnamon',
 'Mast Kalandar',
 'Prasiddhi Food Corner',
 'Gokul Kuteera',
 'Kakaji',
 'Delhi Home Food',
 'Punjabi Dawat']

In [10]:
#encode the input variables
def Encode(zomato):
    for column in zomato.columns[~zomato.columns.isin(['rate', 'cost', 'votes'])]:
        zomato[column] = zomato[column].factorize()[0]
    return zomato
zomato_en = Encode(zomato.copy())


#defining the independent variables
x = zomato_en.iloc[:,[2,4,6,7,8,12]]
y = zomato_en['cost']

#getting Test and Trainnig Set
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.3,random_state=353)
print(x_train)
print(y_train)

       online_order  rate  location  rest_type  cuisines  type
41191             1   3.7        25         32       237     5
1572              1   3.8        11         35        20     4
4743              1   3.0        35         10        69     3
21832             0   3.4        21          2       416     2
30729             1   4.0        19         14       815     4
...             ...   ...       ...        ...       ...   ...
5051              1   3.9        39          0      1009     4
36532             0   4.2        28          0      1718     4
38364             1   3.8        17          7        26     3
17035             0   3.3        11          2        63     4
32381             0   4.6        24         25       178     2

[28865 rows x 6 columns]
41191    1700.0
1572      700.0
4743      150.0
21832     200.0
30729    2000.0
          ...  
5051      800.0
36532    1400.0
38364     350.0
17035     350.0
32381     400.0
Name: cost, Length: 28865, dtype: float64


In [11]:
#Preparing Extra Tree Regression
from sklearn.ensemble import  ExtraTreesRegressor
ETree=ExtraTreesRegressor(n_estimators = 100)
ETree.fit(x_train,y_train)
y_predict=ETree.predict(x_test)
print(y_predict)
from sklearn.metrics import r2_score
r2_score(y_test,y_predict)

[ 311.5  331.   658.5 ...  450.   900.  1100. ]


0.9564472186746422

In [12]:
import random
def predict_rate(l1):
    dfz1 = pd.DataFrame()
    for j in l1:
        v=zomato[zomato['name'] == j].index
        zomato[zomato['name'] == j].index
        z=[]
        for i in v:
            z.append(dict(zomato_en.iloc[i]))
            x=dict(zomato_en.iloc[i])
            dfz1 = dfz1.append(x, ignore_index=True)
    l = dfz1.iloc[:,[2,4,6,7,8,12]]
    x=ETree.predict(l)
    return (sum(x) / len(x))
l2=[]
z=random.randint(0, zomato.shape[0])
l2.append(zomato.iloc[z]['name'])
z=random.randint(0, zomato.shape[0])
l2.append(zomato.iloc[z]['name'])
z=random.randint(0, zomato.shape[0])
l2.append(zomato.iloc[z]['name'])
print(l2)
y=predict_rate(l2)
print(y)

['New Udupi Upahar', 'Chullah Bhatti', 'Willys Top Cafe']
531.046875


In [13]:
#defining the independent variables
x2 = zomato_en.iloc[:,[4,5,6]]
y2 = zomato_en['cuisines']
#getting Test and Trainnig Set
x2_train,x2_test,y2_train,y2_test=train_test_split(x2,y2,test_size=.3,random_state=353)
print(x2_train)
print(y2_train)

       rate  votes  location
41191   3.7     95        25
1572    3.8     69        11
4743    3.0     31        35
21832   3.4      6        21
30729   4.0     53        19
...     ...    ...       ...
5051    3.9    134        39
36532   4.2   3307        28
38364   3.8     33        17
17035   3.3     15        11
32381   4.6    121        24

[28865 rows x 3 columns]
41191     237
1572       20
4743       69
21832     416
30729     815
         ... 
5051     1009
36532    1718
38364      26
17035      63
32381     178
Name: cuisines, Length: 28865, dtype: int64


In [14]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)
#Fit the model
knn.fit(x2_train, y2_train)
test_accuracy = knn.score(x2_test, y2_test)
y_pred = knn.predict(x2_test)
print(test_accuracy)
print(y_pred)

0.6046718396378921
[  67 1541 2132 ...  970 1270  837]


In [15]:
def CountFrequency(my_list):
 
    # Creating an empty dictionary
    freq = {}
    for item in my_list:
        if (item in freq):
            freq[item] += 1
        else:
            freq[item] = 1
    return freq

In [16]:
from statistics import mode
def predict_cuisine(l1):
    cus=""
    dfz1 = pd.DataFrame()
    for j in l1:
        v=zomato[zomato['location'] == j].index
        zomato[zomato['location'] == j].index
        z=[]
        for i in v:
            z.append(dict(zomato_en.iloc[i]))
            x=dict(zomato_en.iloc[i])
            dfz1 = dfz1.append(x, ignore_index=True)
    l = dfz1.iloc[:,[4,5,6]]
    x=knn.predict(l)
    y=mode(x)
    v=zomato_en[zomato_en['cuisines'] == y].index
    s=zomato.iloc[v[0]]
    cus=s['cuisines']
    return [cus,y]
l2=[]
z=random.randint(0, zomato.shape[0])
l2.append(zomato.iloc[z]['location'])
print(l2)
y=predict_cuisine(l2)
print(y)

['Indiranagar']
['North Indian', 5]


In [17]:
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
x3 = zomato_en.iloc[:,[8,9]]
y3 = zomato_en['name']
#getting Test and Trainnig Set
x3_train,x3_test,y3_train,y3_test=train_test_split(x3,y3,test_size=.3,random_state=353)
print(x3_train)
print(y3_train)
gnb = GaussianNB()
gnb.fit(x3_train, y3_train)
y3_predict=gnb.predict(x3_test)

       cuisines    cost
41191       237  1700.0
1572         20   700.0
4743         69   150.0
21832       416   200.0
30729       815  2000.0
...         ...     ...
5051       1009   800.0
36532      1718  1400.0
38364        26   350.0
17035        63   350.0
32381       178   400.0

[28865 rows x 2 columns]
41191    6526
1572      499
4743     2270
21832     833
30729    1858
         ... 
5051     2250
36532    1404
38364    5468
17035     761
32381    2584
Name: name, Length: 28865, dtype: int64


In [18]:
def predict_cuisines1(l1):
    cus=""
    dfz1 = pd.DataFrame()
    for j in l1:
        v=zomato[zomato['location'] == j].index
        zomato[zomato['location'] == j].index
        z=[]
        for i in v:
            z.append(dict(zomato_en.iloc[i]))
            x=dict(zomato_en.iloc[i])
            dfz1 = dfz1.append(x, ignore_index=True)
    l = dfz1.iloc[:,[4,5,6]]
    x=knn.predict(l)
    z=CountFrequency(x)
    z1=sorted(z.items(), key=lambda x:x[1])
    y=mode(x)
    v1=zomato_en[zomato_en['cuisines'] == z1[0][0]].index
    s1=zomato.iloc[v1[0]]
    cus1=s1['cuisines']
    tempcus1=cus1.split(',')
    fcus1=tempcus1[0]
    v2=zomato_en[zomato_en['cuisines'] == z1[1][0]].index
    s2=zomato.iloc[v2[0]]
    cus2=s2['cuisines']
    tempcus2=cus2.split(',')
    fcus2=tempcus2[0]
    v3=zomato_en[zomato_en['cuisines'] == z1[0][0]].index
    s3=zomato.iloc[v3[0]]
    cus3=s3['cuisines']
    tempcus3=cus3.split(',')
    fcus3=tempcus3[0]
    return [(cus1,z1[0][0]),(cus2,z1[1][0]),(cus3,z1[2][0])]
l2=[]
z=random.randint(0, zomato.shape[0])
l2.append(zomato.iloc[z]['location'])
print(l2)
y=predict_cuisines1(l2)
print(y)

['HBR Layout']
[('South Indian, North Indian', 3), ('Continental, Burger, Salad, Pizza, Beverages', 1912), ('South Indian, North Indian', 5)]


In [19]:
def final():
    l2=[]
    z=random.randint(0, zomato.shape[0])
    l2.append(zomato.iloc[z]['name'])
    z=random.randint(0, zomato.shape[0])
    l2.append(zomato.iloc[z]['name'])
    z=random.randint(0, zomato.shape[0])
    l2.append(zomato.iloc[z]['name'])
    print(l2)
    y=predict_rate(l2)
    print(y)
    l2=[]
    z=random.randint(0, zomato.shape[0])
    l2.append(zomato.iloc[z]['location'])
    print(l2)
    z=predict_cuisines1(l2)
    data = [{'cost': y-250,'cuisines': z[0][1]},{'cost': y-250,'cuisines': z[1][1]},{'cost': y-250,'cuisines': z[2][1]},{'cost': y,'cuisines': z[0][1]},{'cost': y,'cuisines': z[1][1]},{'cost': y,'cuisines': z[2][1]},{'cost': y+250,'cuisines': z[0][1]},{'cost': y+250,'cuisines': z[1][1]},{'cost': y+250,'cuisines': z[2][1]}]
    df5 = pd.DataFrame(data)
    f=gnb.predict(df5)
    ff=[]
    print("FEATURED RESTUARANTS ")
    for i in f:
        v=zomato_en[zomato_en['name'] == i].index
        s=zomato.iloc[v[0]]
        cus=s['name']
        ff.append(cus)
        print(cus)
    return [ff[3],ff[4],ff[5]]


In [20]:
def toprest():
    x=final()
    l1=recommend(x[0])
    l2=recommend(x[1])
    l3=recommend(x[2])
    l=l1+l2+l3
    list_set = set(l)
    unique_list = (list(list_set))
    print(" FEW MORE RESTUARANTS ")
    for i in unique_list:
        print(i)
toprest()

['Toast & Tonic', 'Eatzgreen', 'Desi Vdesi']
468.75
['Old Airport Road']
FEATURED RESTUARANTS 
Maravanthe
Juice Berg
Roll Over
Caffe Pascucci
36Th Cross Coffee Mane
Stoner
Rajdhani Thali Restaurant
Fujian Express
Tata Cha
 FEW MORE RESTUARANTS 
The Pizza Bakery
36Th Cross Coffee Mane
Cafe Aladdin
Chez Mariannick
Chikkanna Tiffin Room
Smally'S Resto Cafe
Sidewalk Cafe - Nahar'S Heritage Hotel
D'7 Feast Resto Cafe
Mysuru Coffee Thindi
Kakal-Kai Ruchi
Enchanted Garden
Nutty Squirrel
Prasiddhi Food Corner
On The Nose
Hoot
Burma Burma
Snow'Ora
Mtr 1924
Mayura Sagar
Red Chilli
Ciclo Cafe
Communiti
Mavalli Tiffin Room (Mtr)
Cafe Mondo
Cafe @ Elanza
Pasta Street
New Shanthi Sagar
Onesta
Crawl Street
Stoner
Real Fresh Dosa Corner
Vidyarthi Bhavan
Janahaar
Spice Affair
Podi Idly
Nys Kitchen
Whooppeezz
Caffe Pascucci
